Goal of this notebook:

Tuning the n parameter for IGAR attribute selection algorithm:

Average number of attributes among the projects: 169.16

Minimum number of attributes among the projects: 84


## Tuning strategy

### Collect accuracy for each n

- Vary the n parameter from 1 to 84 (min attributes)
- For each n:
	- For each project:
		- Execute the IGAR using n
		- Execute the prediction (calculate accuracy and normalized improvement)


### Calculate the number of wins for each n:

- For each project:
	- Check which n achieved the best accuracy

IGAR reference: https://www.cs.waikato.ac.nz/~mhall/HallHolmesTKDE.pdf

In [1]:
import numpy as np
import pandas as pd
from IPython.core.display import display

pd.set_option('display.max_columns', None)
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFECV
import warnings
import classifier_utils
warnings.filterwarnings("ignore", category=Warning)
import configs

In [2]:
non_features_columns = ["chunk_id", "line_start", "line_end", "line_separator", "kind_conflict", "url", "project"]
non_features_columns.extend(["project_user", "project_name", "path", "file_name", "sha", "leftsha", "rightsha", "basesha"])

In [3]:
selected_dataset = pd.read_csv("../../data/SELECTED_LABELLED_DATASET.csv")
projects = list(selected_dataset['project'].unique())

In [4]:
rf = RandomForestClassifier(random_state=99, n_jobs=5, n_estimators=100, max_features=0.3, min_samples_leaf=1)

In [5]:
min_n = 0 # min_n = 0 means no selection at all, used as a baseline
max_n = 84

In [6]:
import importlib
importlib.reload(classifier_utils)
tuning_results = classifier_utils.IGAR_tuning(rf, projects, non_features_columns, min_n, max_n, True)

In [7]:
tuning_results

,project,n,accuracy,accuracy_selected,orig_attr
0,Ramblurr__Anki-Android,0,0.727315,0.727315,129
1,apache__directory-server,0,0.935577,0.935577,96
2,android__platform_frameworks_base,0,0.815071,0.815071,566
3,freenet__fred,0,0.671890,0.671890,134
4,alexo__wro4j,0,0.585513,0.585513,107
...,...,...,...,...,...
1695,apache__accumulo,84,0.863732,0.859918,137
1696,Unidata__thredds,84,0.922218,0.911713,99
1697,jgralab__jgralab,84,0.866270,0.863515,98
1698,sebastianbenz__Jnario,84,0.999021,0.999021,84


In [8]:
import importlib
importlib.reload(classifier_utils)
summary = classifier_utils.compute_IGAR_tuning_summary(tuning_results, min_n, max_n)

In [9]:
summary

,n,average_default_accuracy,average_accuracy,improvement,average_ranking,number_wins
0,0,0.800949,0.800949,0.000000,22.925,4
1,1,0.800949,0.686344,-0.143087,72.525,1
2,2,0.800949,0.717579,-0.104088,73.600,0
3,3,0.800949,0.721826,-0.098786,73.950,0
4,4,0.800949,0.729696,-0.088961,73.900,0
...,...,...,...,...,...,...
80,80,0.800949,0.800307,-0.000802,18.375,0
81,81,0.800949,0.800139,-0.001011,18.425,1
82,82,0.800949,0.798367,-0.003223,23.850,1
83,83,0.800949,0.800334,-0.000768,16.100,1


In [10]:
summary.sort_values(['number_wins', 'average_accuracy', 'average_ranking'], ascending=False)

,n,average_default_accuracy,average_accuracy,improvement,average_ranking,number_wins
0,0,0.800949,0.800949,0.000000,22.925,4
70,70,0.800949,0.798830,-0.002646,24.275,2
56,56,0.800949,0.797727,-0.004023,28.400,2
83,83,0.800949,0.800334,-0.000768,16.100,1
81,81,0.800949,0.800139,-0.001011,18.425,1
...,...,...,...,...,...,...
6,6,0.800949,0.742005,-0.073592,71.350,0
5,5,0.800949,0.730545,-0.087900,73.200,0
4,4,0.800949,0.729696,-0.088961,73.900,0
3,3,0.800949,0.721826,-0.098786,73.950,0


In [11]:
summary.sort_values(['average_ranking', 'number_wins', 'average_accuracy'], ascending=True)

,n,average_default_accuracy,average_accuracy,improvement,average_ranking,number_wins
83,83,0.800949,0.800334,-0.000768,16.100,1
73,73,0.800949,0.799399,-0.001935,16.125,0
80,80,0.800949,0.800307,-0.000802,18.375,0
81,81,0.800949,0.800139,-0.001011,18.425,1
68,68,0.800949,0.799208,-0.002174,21.925,0
...,...,...,...,...,...,...
1,1,0.800949,0.686344,-0.143087,72.525,1
5,5,0.800949,0.730545,-0.087900,73.200,0
2,2,0.800949,0.717579,-0.104088,73.600,0
4,4,0.800949,0.729696,-0.088961,73.900,0
